In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, resample
import random as rd
import os
from tqdm import tqdm

# Función para cargar datos desde un archivo binario
def cargar_datos(path):
    """Carga datos desde un archivo binario y crea un DataFrame."""
    with open(path, "rb") as f:
        data = f.read()
    floats = np.frombuffer(data, dtype='<f8')
    samples = floats.reshape(-1, 4)
    df = pd.DataFrame(samples, columns=["SIGNAL", "PICOS", "ENV", "EVENTS"])
    return df

def filtro_pasabajas(data, cutoff_freq, sampling_rate, order=5):
    """Aplica un filtro pasabajas Butterworth a los datos."""
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normalized_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def generar_muestra(env_original, fs_base, sample_id):
    """Genera una muestra con episodios de apnea para entrenamiento de redes neuronales."""
    # Definir parámetros con variación aleatoria
    segundo_inicio = rd.randint(50, 500)  # Inicio aleatorio entre 50 y 750 segundos
    
    # Duraciones de los segmentos (en segundos)
    duraciones = [
        rd.randint(20, 45),    # Normal
        rd.randint(30, 60),    # Hiperpnea
        rd.randint(20, 35),    # Depresión
        rd.randint(8, 20),     # Apnea - Duración más variable
        rd.randint(25, 40)     # Recuperación
    ]
    
    # Verificar que la muestra completa no exceda los 800 segundos
    duracion_total = sum(duraciones)
    if segundo_inicio + duracion_total > 800:
        # Ajustar duraciones proporcionalmente
        factor_ajuste = (800 - segundo_inicio) / duracion_total
        duraciones = [int(d * factor_ajuste) for d in duraciones]
        # Asegurar duraciones mínimas
        duraciones = [max(d, 5) for d in duraciones]
    
    # Factores de escalamiento para la frecuencia respiratoria
    factor_fbs = round(rd.uniform(0.9, 2.0), 2)
    factor_hiper = factor_fbs * round(rd.uniform(1.0, 1.7), 2)
    factor_hipo = factor_fbs * round(rd.uniform(0.5, 0.8), 2)
    factor_recu = factor_fbs * round(rd.uniform(0.8, 1.5), 2)
    factores_fs = [factor_fbs, factor_hiper, factor_hipo, 1.0, factor_recu]
    
    # Frecuencia de muestreo para la visualización y procesamiento
    fs_nuevo = 10  # Hz
    
    # Inicializar listas para almacenar tiempo y señal remuestreada
    tiempo_total_remuestreado = []
    senal_total_remuestreada = []
    tiempo_actual = segundo_inicio
    
    # Almacenamiento para segmentos
    segmentos_remuestreados = []
    tiempos_segmentos = []
    etiquetas = []  # Para guardar las etiquetas de cada punto (1 para apnea, 0 para no apnea)
    
    # Procesar todos los segmentos
    for i, duracion in enumerate(duraciones):
        factor_fs = factores_fs[i]
        fs_segmento_original = int(fs_base * factor_fs)
        inicio_muestra = int(segundo_inicio * fs_base + sum(duraciones[:i]) * fs_base * np.array(factores_fs[:i]).mean() if i > 0 else int(segundo_inicio * fs_base))
        fin_muestra = inicio_muestra + int(duracion * fs_segmento_original)
        
        # Asegurarse de no exceder los límites de la señal original
        if fin_muestra > len(env_original):
            fin_muestra = len(env_original)
        
        env_segmento_original = env_original[inicio_muestra:fin_muestra]
        
        # Remuestrear el segmento a la nueva frecuencia
        num_muestras_nuevo = int(duracion * fs_nuevo)
        if len(env_segmento_original) > 0 and num_muestras_nuevo > 0:
            env_remuestreado = resample(env_segmento_original, num_muestras_nuevo)
            
            # Aplicar factor de escala para el segmento de apnea (índice 3)
            if i == 3:  # Segmento de Apnea
                factor_apnea = rd.uniform(0.000001, 0.01)
                env_remuestreado = env_remuestreado * factor_apnea
                
            tiempo_segmento_nuevo = np.linspace(tiempo_actual, tiempo_actual + duracion, num_muestras_nuevo)
            
            segmentos_remuestreados.append(env_remuestreado)
            tiempos_segmentos.append(tiempo_segmento_nuevo)
            
            # Añadir etiquetas (1 para apnea, 0 para no apnea)
            etiquetas_segmento = np.zeros(num_muestras_nuevo)
            if i == 3:  # Apnea
                etiquetas_segmento = np.ones(num_muestras_nuevo)
                
            etiquetas.extend(etiquetas_segmento)
            
        tiempo_actual += duracion
    
    # Ajustar bordes para garantizar continuidad
    for i in range(len(segmentos_remuestreados)):
        if i > 0:
            # Ajustar el primer valor del segmento actual para que coincida con el último del anterior
            valor_inicial = segmentos_remuestreados[i-1][-1]
            
            # Aplicar una transición suave entre segmentos
            puntos_transicion = min(5, len(segmentos_remuestreados[i]))
            for j in range(puntos_transicion):
                # Interpolación lineal para crear una transición suave
                factor = j / puntos_transicion
                segmentos_remuestreados[i][j] = valor_inicial * (1 - factor) + segmentos_remuestreados[i][j] * factor
    
    # Unir todos los segmentos en una señal continua
    for i, (tiempo, senal) in enumerate(zip(tiempos_segmentos, segmentos_remuestreados)):
        tiempo_total_remuestreado.extend(tiempo)
        senal_total_remuestreada.extend(senal)
    
    # Aplicar un filtro pasabajas suave para eliminar cualquier discontinuidad restante
    senal_total_remuestreada = filtro_pasabajas(senal_total_remuestreada, 2.0, fs_nuevo)
    
    # Crear un DataFrame con la señal y las etiquetas
    df_muestra = pd.DataFrame({
        'tiempo': tiempo_total_remuestreado,
        'senal': senal_total_remuestreada,
        'apnea': etiquetas
    })
    
    return df_muestra

def plot_muestra(df_muestra, sample_id):
    """Visualiza una muestra generada con la etiqueta de apnea resaltada."""
    plt.figure(figsize=(16, 6))
    
    # Graficar la señal
    plt.plot(df_muestra['tiempo'], df_muestra['senal'], 'b-', label='Señal ENV')
    
    # Resaltar las regiones de apnea
    apnea_regions = df_muestra[df_muestra['apnea'] == 1]
    if not apnea_regions.empty:
        plt.fill_between(apnea_regions['tiempo'], apnea_regions['senal'].min(), apnea_regions['senal'].max(),
                         color='red', alpha=0.3, label='Apnea')
    
    plt.title(f"Muestra {sample_id}: Señal ENV con Episodios de Apnea")
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Valor de ENV")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    
    # Guardar la figura
    output_dir = "muestras_visualizacion"
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(f"{output_dir}/muestra_{sample_id}.png")
    plt.close()

# Función principal para generar y guardar muestras
def generar_dataset(path_datos, num_muestras=5000):
    """Genera un conjunto de datos con episodios de apnea para entrenamiento."""
    # Cargar los datos originales
    df = cargar_datos(path_datos)
    env_original = df['ENV'].values
    fs_base = 1024  # Frecuencia de muestreo base (Hz)
    
    # Crear directorios para guardar los datos
    output_dir = "dataset_apnea"
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs("muestras_visualizacion", exist_ok=True)
    
    # Lista para almacenar todas las muestras
    todas_muestras = []
    
    # Generar las muestras con barra de progreso
    for i in tqdm(range(num_muestras), desc="Generando muestras"):
        df_muestra = generar_muestra(env_original, fs_base, i+1)
        
        # Guardar la muestra individual
        df_muestra.to_csv(f"{output_dir}/muestra_{i+1}.csv", index=False)
        todas_muestras.append(df_muestra)
        
        # Visualizar algunas muestras (por ejemplo, cada 500)
        if (i+1) % 500 == 0 or i < 5:
            plot_muestra(df_muestra, i+1)
    
    # Combinar todas las muestras en un único dataset para entrenamiento
    df_final = pd.concat(todas_muestras, keys=range(1, num_muestras+1))
    df_final.to_csv(f"{output_dir}/dataset_completo.csv", index_label=['muestra_id', 'punto_id'])
    
    print(f"Generadas {num_muestras} muestras en '{output_dir}'")
    print(f"Algunas visualizaciones guardadas en 'muestras_visualizacion'")
    
    # Estadísticas del dataset
    num_puntos_apnea = sum(df['apnea'].sum() for df in todas_muestras)
    num_puntos_total = sum(len(df) for df in todas_muestras)
    porcentaje_apnea = (num_puntos_apnea / num_puntos_total) * 100
    
    print(f"Estadísticas del dataset:")
    print(f"- Total de puntos: {num_puntos_total}")
    print(f"- Puntos con apnea: {num_puntos_apnea} ({porcentaje_apnea:.2f}%)")
    print(f"- Puntos sin apnea: {num_puntos_total - num_puntos_apnea} ({100 - porcentaje_apnea:.2f}%)")

if __name__ == "__main__":
    # Ruta del archivo de datos (actualizar según sea necesario)
    path = "P/P1e.dat"
    
    # Generar el dataset
    generar_dataset(path, num_muestras=5000)

Generando muestras: 100%|██████████| 5000/5000 [02:51<00:00, 29.16it/s]


Generadas 5000 muestras en 'dataset_apnea'
Algunas visualizaciones guardadas en 'muestras_visualizacion'
Estadísticas del dataset:
- Total de puntos: 7576160
- Puntos con apnea: 698540.0 (9.22%)
- Puntos sin apnea: 6877620.0 (90.78%)
